<a href="https://colab.research.google.com/github/SaswatD27/21cs460_group01/blob/main/Code/Subclustering_P2P_OldFramework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time

import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import DBSCAN
import os

In [ ]:
def get_data(m, n):  # Generates data with linear trend
    x_cord = []
    y_cord = []
    for i in range(n):
        x = np.random.rand(w_n)
        y = np.dot(m, x)
        x_cord.append(x)
        y_cord.append(y)
    return x_cord, y_cord


def data_1():
    m = m1
    # print(m)
    x, y = get_data(m, sum(n_k_dev))
    i = 0
    x_cord = []
    y_cord = []
    for n0 in n_k_dev:
        x_cord.append(x[i:i + n0])
        y_cord.append(y[i:i + n0])
        i += n0
    return x_cord, y_cord


def test_data1():
    m = m1
    testing_x, testing_y = get_data(m, 100)
    return testing_x, testing_y


def data_2():
    m = m2
    # print(m)
    x, y = get_data(m, sum(n_k_dev))
    i = 0
    x_cord = []
    y_cord = []
    for n0 in n_k_dev:
        x_cord.append(x[i:i + n0])
        y_cord.append(y[i:i + n0])
        i += n0
    return x_cord, y_cord


def test_data2():
    m = m2
    testing_x, testing_y = get_data(m, 100)
    return testing_x, testing_y


def data_3():
    m = m3
    # print(m)
    x, y = get_data(m, sum(n_k_dev))
    i = 0
    x_cord = []
    y_cord = []
    for n0 in n_k_dev:
        x_cord.append(x[i:i + n0])
        y_cord.append(y[i:i + n0])
        i += n0
    return x_cord, y_cord


def test_data3():
    m = m3
    testing_x, testing_y = get_data(m, 100)
    return testing_x, testing_y

In [ ]:
def SGD_(m, n, iteration, x_cord, y_cord, alpha):  # Performs client side SGD
    for p in range(iteration):
        for i in range(n):
            j = random.randint(0, n - 1)
            x_i = x_cord[j]
            y_a = y_cord[j]
            y_p = np.dot(m, x_i)
            dm = 2 * x_i * (y_p - y_a)
            m = m - alpha * dm
    return m


def no_of_pts(id):
    return n_k_dev[id]


def fetch_coords(id, x_cord, y_cord):
    return x_cord[id], y_cord[id]


def device_local_learning(alp_, w, id, itern):
    # print("Device %i running" % id)
    n_k = no_of_pts(id)
    x_cord, y_cord = data_1()
    x_cord, y_cord = fetch_coords(id, x_cord, y_cord)
    w = SGD_(w, n_k, itern, x_cord, y_cord, alp_)
    save_weight(w, n_k, id)
    return w, n_k


def device_local_learning_2(alp_, w, id, itern):
    # print("Device %i running" % id)
    n_k = no_of_pts(id)
    x_cord, y_cord = data_2()
    x_cord, y_cord = fetch_coords(id, x_cord, y_cord)
    w = SGD_(w, n_k, itern, x_cord, y_cord, alp_)
    save_weight(w, n_k, id)
    return w, n_k


# Conducts local training for SGD and outputs weight vector
def device_local_learning_3(alp_, w, id, itern):
    # print("Device %i running" % id)
    n_k = no_of_pts(id)
    x_cord, y_cord = data_3()
    x_cord, y_cord = fetch_coords(id, x_cord, y_cord)
    w = SGD_(w, n_k, itern, x_cord, y_cord, alp_)
    save_weight(w, n_k, id)
    return w, n_k

In [ ]:
def federated_avg(w_, n_k):
    n_ = np.sum(n_k)
    w_k = 0
    for p in range(len(n_k)):
        w_k += (n_k[p] * w_[p]) / n_
    return w_k


def SGD_federated_learning_new(T, K, C, w, n, alp_):
    T1 = T
    w_ = np.zeros((int(C * K), w_n))
    n_k = np.zeros(int(C * K))
    prob = 0.1
    prob2 = 0.15
    # Train for T many rounds
    for i in range(T1):
        training_subset = np.random.choice(all_device, int(C * K), replace=False)
        # training_subset = np.random.randint(0, K, int(C * K))
        z = 0
        for j in training_subset:
            ran = random.random()
            if ran < prob:
                w_[z], n_k[z] = device_local_learning(alp_, w, j, n)
            elif ran > prob and ran < prob2:
                w_[z], n_k[z] = device_local_learning_2(alp_, w, j, n)
            else:
                w_[z], n_k[z] = device_local_learning_3(alp_, w, j, n)
            z += 1
        w_k = federated_avg(w_, n_k)
        w = w_k
        print("Global Iteration ", i, " done.")


    print("Initial Model sent.")
    w_global = w
    w_k, device_no = new_update(w_, training_subset, n_k, alp_, n, T)
    return w_k, w_global, device_no


def new_update(w_, training_subset, n_k, alp_, n, T):
    w_mult = []
    clustering = DBSCAN(eps=0.5, min_samples=1).fit(w_)
    print(clustering.labels_)
    device_number = []
    device_nu=0
    for i in range(np.max(clustering.labels_) + 1):
        training_subset_n = []
        n_k_n = []
        w_new = []
        for k in range(len(training_subset)):
            lab = clustering.labels_[k]
            if i == lab:
                training_subset_n.append(training_subset[k])
                n_k_n.append(n_k[k])
                w_new.append(w_[k])
                device_nu+=1
        device_number.append(len(training_subset_n))
        """wt = federated_avg(w_new, n_k_n)
        n_k_t = np.zeros(len(training_subset_n))
        w_t = np.zeros((len(training_subset_n), w_n))"""
        for j in range(T):
            initiate = np.random.choice(training_subset_n)
            w_t = p2p_learn(initiate, training_subset_n, i)
            print("Class ", i, " Iteration ", j, " done.")
        """for i in training_subset_n:
            i = str(i)
            os.remove(i + "_wn.dat")"""
        w_mult.append(w_t)
        print(device_number,device_nu)
    return w_mult, device_number

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]


def average_testing_error(w, testing_x, testing_y):
    error = 0
    for i in range(len(testing_x)):
        y_p = np.dot(w, testing_x[i])
        y_a = testing_y[i]
        # print(i)
        error += (y_a - y_p) ** 2
    error *= (1 / len(testing_x))
    return error

In [ ]:
def request_dat(initiate):
    fl = str(initiate)
    w_n = np.loadtxt(fl + "_wn.dat")
    n_k_new = n_k_devices[initiate]
    return w_n, n_k_new


def p2p_learn(initate, training_subset, clas):
    w_n, n_k_new = request_dat(initate)
    training_subset_c = training_subset.copy()
    training_subset_c.remove(initate)
    w_new = communicate(w_n, training_subset_c, n_k_new, clas)
    return w_new


def communicate(w_o, traiinng_subset, n_k, clas):
    print(traiinng_subset)
    comm_len = np.random.randint(1, len(traiinng_subset))
    call_to = np.random.choice(traiinng_subset, comm_len)
    print("Call", call_to)
    w_ = []
    n_ = []
    w_.append(w_o)
    n_.append(n_k)
    for i in call_to:
        w_w, n_w = request_dat(i)
        w_.append(w_w)
        n_.append(n_w)
    w_new = federated_avg(w_, n_)
    
    if clas == 0:
        testing_x1, testing_y1 = test_data1()
        err_o = average_testing_error(w_o, testing_x1, testing_y1)
        err_1 = average_testing_error(w_new, testing_x1, testing_y1)
    elif clas == 1:
        testing_x2, testing_y2 = test_data2()
        err_o = average_testing_error(w_o, testing_x2, testing_y2)
        err_1 = average_testing_error(w_new, testing_x2, testing_y2)
    else:
        testing_x3, testing_y3 = test_data3()
        err_o = average_testing_error(w_o, testing_x3, testing_y3)
        err_1 = average_testing_error(w_new, testing_x3, testing_y3)
        
    if err_1 < err_o:
        w_nn = w_new
    else:
        w_nn = w_o

    m=0
    for i in call_to:
        n_k_=n_[m]
        save_weight(w_nn,n_k_,i)
        m+=1

    return w_nn


def save_weight(w, n_k, id):
    n_k_devices[id] = n_k
    fl = str(id)
    file = open(fl + "_wn.dat", "w")
    w_p = [w]
    for row in w_p:
        np.savetxt(file, row)
    file.close()

In [ ]:
m1 = [1.5, 12, 4.0, 2.9, 4.8, 6.1, 1]
m2 = [1, 12, 4, 3, 5, 6, 1.2]
m3 = [1.1, 12.2, 3.9, 3, 4.8, 5.8, 0.7]
k = 500
n_k_dev = np.random.randint(100, 1000, k)
w_n = 7  # cardinality of weight vector

T = 10  # No of federated learning rounds
K = 500  # Total no of nodes
C = 0.25  # fraction of nodes used at each iteration
# B = #Local batch size used at each learning iteration
# random Model
w = np.random.rand(w_n)
n = 50  # number of iteration for local training before pooling
alp_ = 0.01  # local learning rate
n_k = random.randint(10, 100)
n_k_devices = np.zeros(K)
all_device = np.arange(K)
# x_cord1, y_cord1 = get_data(m, n_k)

time1 = time.time()
w1, w_global, device_no = SGD_federated_learning_new(T, K, C, w, n, alp_)
time2 = time.time()
print("Time taken for SGD federated learning", time2 - time1)

Global Iteration  0  done.
Global Iteration  1  done.
Global Iteration  2  done.
Global Iteration  3  done.
Global Iteration  4  done.
Global Iteration  5  done.
Global Iteration  6  done.
Global Iteration  7  done.
Global Iteration  8  done.
Global Iteration  9  done.
Initial Model sent.
[0 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0
 0 0 0 0 0 0 1 2 0 0 0 0 0 2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 2 0 0 2 0 0 0 0 0 0 2 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
[30, 3, 495, 337, 36, 142, 416, 496, 49, 341, 150, 292, 33, 41, 403, 422, 371, 449, 273, 339, 48, 305, 390, 160, 134, 399, 478, 334, 183, 252, 91, 259, 420, 415, 465, 405, 489, 395, 473, 147, 444, 152, 108, 407, 35, 201, 280, 497, 466, 54, 476, 494, 51, 222, 330, 349, 217, 61, 97, 96, 307, 401, 42, 294, 315, 192, 385, 185, 376, 467, 236, 345, 15, 332, 149, 256, 119, 227, 428, 22, 4, 8, 302, 445, 398, 396, 429, 214, 138, 161, 417, 378, 144, 94, 278, 38

In [ ]:
print("Original weigts for minority(I) is ", m1, " and minority(II) ", m2, " and majority is:", m3)
print("Global model has weights:", w_global)
print("Classed multi model has weights:", w1)
print("Clustering number", device_no)

Original weigts for minority(I) is  [1.5, 12, 4.0, 2.9, 4.8, 6.1, 1]  and minority(II)  [1, 12, 4, 3, 5, 6, 1.2]  and majority is: [1.1, 12.2, 3.9, 3, 4.8, 5.8, 0.7]
Global model has weights: [ 1.12207832 12.17756961  3.91121507  2.99334128  4.80911299  5.82908901
  0.74275851]
Classed multi model has weights: [array([ 1.10000012, 12.1999999 ,  3.89999981,  3.00000002,  4.79999998,
        5.80000008,  0.7000001 ]), array([ 1.49999859, 11.99999938,  4.0000002 ,  2.9000017 ,  4.80000115,
        6.09999947,  0.9999996 ]), array([ 0.99999999, 12.00000002,  3.99999999,  3.00000004,  4.99999997,
        6.00000003,  1.19999995])]
Clustering number [110, 8, 7]


In [ ]:
m_1 = w1[0]
m_2 = w1[1]
m_3 = w1[2]
testing_x1, testing_y1 = test_data1()
testing_x2, testing_y2 = test_data2()
testing_x3, testing_y3 = test_data3()
print("Total number of device = ", int(K * C))
print("Cluster having the majority has %i devices." % device_no[0])
print("Cluster having the minority has %i devices and %i devices." % (device_no[1], device_no[2]))
print("Average Testing Error for Global model:\n For majority devices error =",
      average_testing_error(w_global, testing_x3, testing_y3), "\n For minority cluster with ", device_no[1],
      " devices has error =", average_testing_error(w_global, testing_x1, testing_y1), "\n For minority cluster with ",
      device_no[2], "devices error =", average_testing_error(w_global, testing_x2, testing_y2))

print("Average Testing Error for Separate classified model:\n For majority devices error",
      average_testing_error(m_1, testing_x3, testing_y3), "\n For minority cluster with ", device_no[1],
      " devices has error =", average_testing_error(m_2, testing_x1, testing_y1), "\n For minority cluster with ",
      device_no[2], "devices error =", average_testing_error(m_3, testing_x2, testing_y2))

Total number of device =  125
Cluster having the majority has 110 devices.
Cluster having the minority has 8 devices and 7 devices.
Average Testing Error for Global model:
 For majority devices error = 0.0020340558356823184 
 For minority cluster with  8  devices has error = 0.15473696385763466 
 For minority cluster with  7 devices error = 0.1213038031084982
Average Testing Error for Separate classified model:
 For majority devices error 6.719255435575832e-15 
 For minority cluster with  8  devices has error = 6.302195490774651e-13 
 For minority cluster with  7 devices error = 6.599850237706375e-16
